# Load needed modules

In [1]:
# modules:
import pandas as pd
import os
from openai import OpenAI


# get external variables or functions:
import src.API_key as key

# Load data to translate
> this should be the drawn concept data set whereby 1 row is 1 drawn concept

In [2]:
print(os.getcwd())
# Get the current working directory
os.chdir("data") 
directory = os.getcwd()

c:\Users\fenn\Nextcloud\Artikel Cognitive Affective Maps Tools\CAM tools\aaa_translateCAMs


In [3]:
# print the current working directory
print(directory)

c:\Users\fenn\Nextcloud\Artikel Cognitive Affective Maps Tools\CAM tools\aaa_translateCAMs\data


In [4]:
# List files in the current working directory
files = os.listdir('.')
# Display the list of files
print(files)

['drawnConcepts_translate.xlsx']


In [5]:
# Load the .xlsx file into a DataFrame
df = pd.read_excel(directory + "/drawnConcepts_translate.xlsx")
# Specify the columns to keep
columns_to_keep_text = ['text'] # only the df column 'text' will be kept
columns_to_keep_comment = ['id', 'comment'] # only the df columns 'id' and 'comment' will be kept

# Filter the DataFrame to keep only the specified columns
df_text = df[columns_to_keep_text]
df_comment = df[columns_to_keep_comment]

# prepare the data frames 
# for the text only the unique values are kept
print("Length of the text:")
print(df_text.__len__())
df_text = df_text['text'].str.strip()
df_text = df_text.unique()
print(df_text.__len__())
# for the comment the NaN values are removed
print("Length of the comment:")
print(df_comment.__len__())
df_comment = df_comment.dropna()
print(df_comment.__len__())

Length of the text:
6404
2380
Length of the comment:
6404
1249


In [6]:
print(df_text)

['Autonomiebedenken' 'Unklare Verantwortung' '"Kleine" Operationen' ...
 'zuverlassing' 'Zuverlässligkeit' 'Zwiespalt Kind/Erwachsener']


In [7]:
print(df_comment)

                                        id  \
0     205c7562-cb88-4997-b064-d3a8712dde7f   
2     0eb4922a-4e3d-4bae-8eba-e6112322123c   
4     bb704e1e-1771-4e41-98c9-6f34634d9e88   
6     caff0cfe-11b1-4fd9-a78f-3121ba0d3152   
7     0291051a-64c6-43aa-a870-63c529178835   
...                                    ...   
6373  4a60501f-f8a1-4c35-bcf8-c82f581c1123   
6386  11becfac-bff0-47b6-8a6c-2939cf6885e2   
6389  6b166e65-dc10-43aa-ab96-24a30588ea18   
6392  33d7e5e1-bef4-483e-ac39-c210a6aee480   
6395  2f126d53-318b-4919-91f2-1da95c9b6458   

                                                comment  
0     Überlegung, ob Menschen die Roboter steuern so...  
2                 Über das Handeln von Rettungsrobotern  
4     Besonders gut für kleinere Operationen und mög...  
6                     Organische, süße Assistenzroboter  
7     Personen, die keinen Körperkontakt mit anderen...  
...                                                 ...  
6373                                   is

# Set up ChatGPT

## Translate written text

In [8]:
# Step 1: Set up the OpenAI API
client = OpenAI(
    api_key=key.openai_api_key,
)

# Define the array of German words
words_to_translate = df_text


# Function to translate a word using ChatGPT API
def translate_word(word):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that translates German words to English."},
                {"role": "user", "content": f'Translate the following German word to English: "{word}" Be careful with spelling errors and do not use capital letters for adjectives. Only return the translation in quotes.'}
            ]
        )
        # print(response.choices[0].message.content.strip())
        
        #content = response.choices[0].message.content.strip()
        #start_index = content.find('is "') + 4
        #end_index = content.find('"', start_index)
        #translation = content[start_index:end_index]
        #print(translation)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error translating word '{word}': {e}")
        return None

# Step 2: Translate the words
translations = [translate_word(word) for word in words_to_translate]

# Remove None values if any translation fails
translated_words = [word for word, translation in zip(words_to_translate, translations) if translation is not None]
translations = [translation for translation in translations if translation is not None]

# Step 3: Store the translations in a .xlsx file
df_translated_text = pd.DataFrame({
    'German': translated_words,
    'English': translations
})
print(df_translated_text)

                          German                      English
0              Autonomiebedenken          "autonomy concerns"
1          Unklare Verantwortung     "unclear responsibility"
2           "Kleine" Operationen           "small operations"
3                  "Soft robots"                "soft robots"
4       "Touch starved" Personen     ""touch starved" people"
...                          ...                          ...
2375  Zuverlässiger als Menschen  "more reliable than humans"
2376             Zuverlässigkeit                "reliability"
2377                zuverlassing                   "reliable"
2378            Zuverlässligkeit                "reliability"
2379  Zwiespalt Kind/Erwachsener       "conflict child/adult"

[2380 rows x 2 columns]


In [9]:
print(os.getcwd())

os.chdir("../output") 

# Save the DataFrame to an Excel file
df_translated_text.to_excel('translations_text.xlsx', index=False)
print("Translations saved to translations_text.xlsx")

c:\Users\fenn\Nextcloud\Artikel Cognitive Affective Maps Tools\CAM tools\aaa_translateCAMs\data
Translations saved to translations_text.xlsx


## Translate written comments

In [16]:
print(df_comment['comment'][0:3])
print(df_comment)

0    Überlegung, ob Menschen die Roboter steuern so...
2                Über das Handeln von Rettungsrobotern
4    Besonders gut für kleinere Operationen und mög...
Name: comment, dtype: object
                                        id  \
0     205c7562-cb88-4997-b064-d3a8712dde7f   
2     0eb4922a-4e3d-4bae-8eba-e6112322123c   
4     bb704e1e-1771-4e41-98c9-6f34634d9e88   
6     caff0cfe-11b1-4fd9-a78f-3121ba0d3152   
7     0291051a-64c6-43aa-a870-63c529178835   
...                                    ...   
6373  4a60501f-f8a1-4c35-bcf8-c82f581c1123   
6386  11becfac-bff0-47b6-8a6c-2939cf6885e2   
6389  6b166e65-dc10-43aa-ab96-24a30588ea18   
6392  33d7e5e1-bef4-483e-ac39-c210a6aee480   
6395  2f126d53-318b-4919-91f2-1da95c9b6458   

                                                comment  
0     Überlegung, ob Menschen die Roboter steuern so...  
2                 Über das Handeln von Rettungsrobotern  
4     Besonders gut für kleinere Operationen und mög...  
6                    

In [22]:
print(os.getcwd())

c:\Users\fenn\Nextcloud\Artikel Cognitive Affective Maps Tools\CAM tools\aaa_translateCAMs\output


In [23]:
# Step 1: Set up the OpenAI API
client = OpenAI(
    api_key=key.openai_api_key,
)

# Define the array of German words
comments_to_translate = df_comment['comment']
comments_ids_to_translate = df_comment['id']

# Function to translate a word using ChatGPT API
def translate_comment(comment):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that translates German comments to English."},
                {"role": "user", "content": f'Translate the following German comment to English: "{comment}" Be careful with spelling errors and do not use capital letters for adjectives. Only return the translation in quotes.'}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error translating comment '{comment}': {e}")
        return None

# Step 2: Translate the words with logging and saving
translations = []
counter = 0
save_interval = 50
print_interval = 100

for comment in comments_to_translate:
    translation = translate_comment(comment)
    if translation:
        translations.append(translation)
    counter += 1
    if counter % print_interval == 0:
        print(f"Current counter: {counter}")
    if counter % save_interval == 0:
        df_temp = pd.DataFrame({'ID': comments_ids_to_translate[:counter], 'German': comments_to_translate[:counter], 'English': translations})
        df_temp.to_excel(f'translations_comment_up_to_{counter}.xlsx', index=False)

# Final save
df_translated_comments = pd.DataFrame({
    'ID': comments_ids_to_translate,
    'German': comments_to_translate,
    'English': translations
})
df_translated_comments.to_excel('translations_comment_final.xlsx', index=False)

print(df_translated_comments)


Current counter: 100
Current counter: 200
Current counter: 300
Current counter: 400
Current counter: 500
Current counter: 600
Current counter: 700
Current counter: 800
Current counter: 900
Current counter: 1000
Current counter: 1100
Current counter: 1200
                                        ID  \
0     205c7562-cb88-4997-b064-d3a8712dde7f   
2     0eb4922a-4e3d-4bae-8eba-e6112322123c   
4     bb704e1e-1771-4e41-98c9-6f34634d9e88   
6     caff0cfe-11b1-4fd9-a78f-3121ba0d3152   
7     0291051a-64c6-43aa-a870-63c529178835   
...                                    ...   
6373  4a60501f-f8a1-4c35-bcf8-c82f581c1123   
6386  11becfac-bff0-47b6-8a6c-2939cf6885e2   
6389  6b166e65-dc10-43aa-ab96-24a30588ea18   
6392  33d7e5e1-bef4-483e-ac39-c210a6aee480   
6395  2f126d53-318b-4919-91f2-1da95c9b6458   

                                                 German  \
0     Überlegung, ob Menschen die Roboter steuern so...   
2                 Über das Handeln von Rettungsrobotern   
4     Besonders